In [53]:
import json

In [164]:
import sys
sys.path.append('../utilities/')

In [167]:
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import matplotlib.pyplot as plt
from mmd import MMD
from sklearn.model_selection import train_test_split
from tqdm import tqdm
# import openai

In [26]:
from openai import OpenAI

In [ ]:
import os
from dotenv import load_dotenv

In [6]:
sentence_transformer = SentenceTransformer('all-mpnet-base-v2')

In [9]:
df = pd.read_csv('../../CODsurf/cmb_prosocial_labeled.csv')
player_chats = df['player_chat']
df

,match_id,player_name,player_chat,action
0,22398,JCamz,nice lighting bolt zeus. report all of you. cl...,NOT-PROSOCIAL
1,26872,zua halucynka.,ty,PROSOCIAL
2,24083,cnywad,ggwp,PROSOCIAL
3,47512,StartWithA.BANG,LOL . Ez,NOT-PROSOCIAL
4,22039,Sol'ka,sry. ggwp,PROSOCIAL
...,...,...,...,...
4995,18647,Phantasm,yea. line problem?. valve?. fuck this,NOT-PROSOCIAL
4996,37574,1of3,gl hf. I'm not exe still nice to meet you. gg wp,PROSOCIAL
4997,28742,PookieMarkCow,pls respond. in any kind. including insult. cr...,UNCLEAR
4998,28276,Madara,np. FF. afk,NOT-PROSOCIAL


In [14]:
df_unclear_removed = df[df['action'] != 'UNCLEAR']
df_unclear_removed['action'] = df['action'].replace({'NOT-PROSOCIAL': -1, 'PROSOCIAL': 1})

/var/folders/lv/pnwq6bmj4tq68bsvy__37qyh0000gn/T/ipykernel_20444/1358108983.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unclear_removed['action'] = df['action'].replace({'NOT-PROSOCIAL': -1, 'PROSOCIAL': 1})


# **Split**

In [15]:
train_df, test_df = train_test_split(df_unclear_removed, test_size=0.3)

In [47]:
test_df

,match_id,player_name,player_chat,action
2692,36749,Rakesh,oopps. :D. nab. ez,-1
2417,6525,VeNoM,lol. gg guys wp,1
1791,19062,Meow,lel. just lel. rofl. commend luna. report luna...,-1
2060,21133,HerpyDerpy,tghf. Ez,-1
2983,25496,ihatetryinghardcarries,press J for free predctio. lol. 14mins?. k. fu...,-1
...,...,...,...,...
4068,20452,Chops Buster,gg,1
1214,428,Jason,ggwp. nigga please. ez. ez game,-1
4700,39590,DotP.Fafnir,gg,1
1009,22375,It's not your fault,what seems to be the problem gentlemen. gg,1


In [18]:
train_df

,match_id,player_name,player_chat,action
3593,18727,[B]rix,WP,1
4930,31016,Dominant deer,Ez,-1
134,28082,Сын Собаки,rofl. all on me?. GG,-1
541,27540,DG PUFFMG,bet . BET. BET YOU FUCJERS. to bif hc. dk. gg,-1
2156,27123,#Ex3maL,gg,1
...,...,...,...,...
1655,20318,GIGSY DANGER,GG. RPEORT LEGION. PLS. VERY VERY NOOB AND MIS...,-1
4401,19325,Trickster,outplayed. gg,1
999,27265,manbearpig,GG,1
1175,45137,Doctor is coming,"mne kajetsya u nas kogoto ne hvataaet, tipa ca...",-1


In [44]:
sample = train_df.sample(n=5)
sample

,match_id,player_name,player_chat,action
4995,18647,Phantasm,yea. line problem?. valve?. fuck this,-1
2978,46459,let me entertain u,ez,-1
4526,28928,30minSKADI,WHAWHWAHAHWAW. POGNOBILI. juga. gei ebanie. go...,-1
2572,12664,AnyWayAnyDay,ez. gg,-1
3718,45394,FirstPickSFLEGEND,GO GO 1VS5. rEPORT. vENO. uEBANA,-1


# **Trial ChatGPT API**

Checking the number of input tokens the output would generate

In [ ]:
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=API_KEY)

In [83]:
instruction = (
    "You are a data generator tasked with creating realistic game chat content "
    "based on provided labels. You should maintain the style, slang, and informal tone of the chat, "
    "including typos where appropriate, but the content must not be copied. "
    "Return a list of new entries in valid JSON format, where each entry is a Python dictionary with the keys "
    "'player_chat' and 'action'."
)
input = (
    f"Here is a sample of labeled game chat data:\n\n"
    f"{train_df[['player_chat', 'action']].to_dict(orient='records')}\n\n"
    "Please generate 907 new samples that are similar in style, and output them as a valid JSON list of dictionaries. "
    "Each dictionary should have 'player_chat' as a string and 'action' as a string (either '1' or '-1')."
)

response = client.responses.create(
    model="gpt-4o",
    instructions=instruction,
    input=input
)

In [84]:
print(response.output_text)

```json
[
    {"player_chat": "gg", "action": "1"},
    {"player_chat": "ez money", "action": "-1"},
    {"player_chat": "GG EZ", "action": "1"},
    {"player_chat": "lol noob team", "action": "-1"},
    {"player_chat": "What a match!", "action": "1"},
    {"player_chat": "gg wp guys", "action": "1"},
    {"player_chat": "GET REKT", "action": "-1"},
    {"player_chat": "nice try. gg", "action": "1"},
    {"player_chat": "srsly? report", "action": "-1"},
    {"player_chat": "rofl trash", "action": "-1"},
    {"player_chat": "ggwp", "action": "1"},
    {"player_chat": "gg easy", "action": "-1"},
    {"player_chat": "thx 4 the game!", "action": "1"},
    {"player_chat": "team sucks", "action": "-1"},
    {"player_chat": "it was fun :)", "action": "1"},
    {"player_chat": "noob city", "action": "-1"},
    {"player_chat": "gg well played", "action": "1"},
    {"player_chat": "learn to play", "action": "-1"},
    {"player_chat": "glhf", "action": "1"},
    {"player_chat": "lagggg omg", "act

In [95]:
resp = response.output_text[8:-4]

In [ ]:
data = json.loads(resp)
new_df = pd.DataFrame(data)

In [99]:
new_df.to_csv('../data/generated/train_test_split/large_sample.csv', index=False)

# **Running Comparisons**

In [198]:
new_chats = pd.read_csv('../data/generated/train_test_split/large_sample.csv')

In [199]:
new_chats = new_df['player_chat'].to_list()

In [200]:
X = np.array(sentence_transformer.encode(test_df['player_chat'].to_list()))

In [182]:
X.shape

(907, 768)

In [202]:
Y = np.array(sentence_transformer.encode(new_chats))

In [203]:
def report_MMD(X, Y, normalize=False):
    
    tensorX = torch.tensor(X)
    tensorY = torch.tensor(Y)
    rbf_mmd = MMD(tensorX, tensorY, "rbf")
    scale_mmd = MMD(tensorX, tensorY, "multiscale")


    if normalize:
        return (rbf_mmd.item() / np.sqrt((1.0 / X.shape[0]) + (1.0 / Y.shape[0]))), (scale_mmd.item() / np.sqrt(1.0 / X.shape[0] + 1.0 / Y.shape[0]))
    else:
        return rbf_mmd.item(), scale_mmd.item()

In [215]:
X

array([[-0.03209211,  0.01761368,  0.00489417, ..., -0.00227808,
        -0.05074299, -0.04014027],
       [-0.02820076,  0.05093875, -0.01459465, ...,  0.02868473,
        -0.00579219, -0.05142245],
       [ 0.01110198, -0.08723923,  0.00197649, ...,  0.02278924,
        -0.03066735, -0.0186208 ],
       ...,
       [-0.01783033, -0.02038394,  0.00390915, ...,  0.03911946,
        -0.03475408, -0.00888458],
       [ 0.06343815,  0.005559  ,  0.00664612, ...,  0.06737191,
        -0.01323845,  0.00789483],
       [-0.02423655,  0.01497339,  0.00459325, ...,  0.0288214 ,
         0.00279358, -0.02258008]], dtype=float32)

In [216]:
Y

array([[-0.01783041, -0.02038393,  0.00390918, ...,  0.03911939,
        -0.03475412, -0.00888458],
       [-0.01559118,  0.01486435,  0.01145685, ..., -0.05138317,
        -0.07126011, -0.03618212],
       [-0.01521565, -0.05675112,  0.01965393, ...,  0.00581456,
        -0.06183083, -0.02463908],
       ...,
       [ 0.01978553,  0.02642662, -0.02289566, ...,  0.0390843 ,
        -0.01082369, -0.01480835],
       [ 0.03459815, -0.07308748, -0.01351438, ...,  0.02217147,
         0.00608943, -0.05841101],
       [ 0.02569757, -0.0483445 ,  0.01427542, ...,  0.03355123,
        -0.05215664, -0.03678662]], dtype=float32)

In [204]:
X.shape[0]

907

In [205]:
len(new_chats)

555

In [206]:
rbf, scale = report_MMD(X, Y, normalize=True)

In [207]:
report_MMD(X, Y, normalize=True)[0]

0.49064090687025186

In [178]:
print(rbf)

0.49064090687025186


In [191]:
print(X)

[[-0.03209211  0.01761368  0.00489417 ... -0.00227808 -0.05074299
  -0.04014027]
 [-0.02820076  0.05093875 -0.01459465 ...  0.02868473 -0.00579219
  -0.05142245]
 [ 0.01110198 -0.08723923  0.00197649 ...  0.02278924 -0.03066735
  -0.0186208 ]
 ...
 [-0.01783033 -0.02038394  0.00390915 ...  0.03911946 -0.03475408
  -0.00888458]
 [ 0.06343815  0.005559    0.00664612 ...  0.06737191 -0.01323845
   0.00789483]
 [-0.02423655  0.01497339  0.00459325 ...  0.0288214   0.00279358
  -0.02258008]]


In [192]:
print(Y)

[[-0.01783041 -0.02038393  0.00390918 ...  0.03911939 -0.03475412
  -0.00888458]
 [-0.01559118  0.01486435  0.01145685 ... -0.05138317 -0.07126011
  -0.03618212]
 [-0.01521565 -0.05675112  0.01965393 ...  0.00581456 -0.06183083
  -0.02463908]
 ...
 [ 0.01978553  0.02642662 -0.02289566 ...  0.0390843  -0.01082369
  -0.01480835]
 [ 0.03459815 -0.07308748 -0.01351438 ...  0.02217147  0.00608943
  -0.05841101]
 [ 0.02569757 -0.0483445   0.01427542 ...  0.03355123 -0.05215664
  -0.03678662]]


In [214]:
report_MMD(X, Y, normalize=True)[0]

0.49064090687025186

In [260]:
def p_test(X, Y, obs_mmd, n_obs=1000):
    n = X.shape[0]
    combined = np.vstack([X, Y])
    permuted_mmds = []

    for _ in tqdm(range(n_obs)):
        np.random.shuffle(combined)  
        X_perm = combined[:n]  
        Y_perm = combined[n:]
        # print(X_perm.shape)
        # print(Y_perm.shape)
        permuted_mmd = report_MMD(X_perm, Y_perm, normalize=True)[0]
        # print(permuted_mmd)
        permuted_mmds.append(permuted_mmd)
    
    permuted_mmds = np.array(permuted_mmds)
    mean_mmd = np.mean(permuted_mmds)
    std_mmd = np.std(permuted_mmd)
    p_value = np.mean(permuted_mmds >= obs_mmd)

    return p_value, mean_mmd, std_mmd, permuted_mmds

In [262]:
p_value, x_bar, std, permuted = p_test(X, Y, rbf)
print(p_value)
print(f'{x_bar} ± {std}')

100%|██████████| 1000/1000 [00:34<00:00, 28.92it/s]

0.0
0.009692555726559974 ± 0.0


In [210]:
np.std(permuted)

0.0019838068263262464

In [196]:
np.mean(permuted)

0.00976435748229921

# **Prosocial Cateogry Generations**

In [225]:
prosocial_train_chats = train_df[train_df['action'] == 1]

In [226]:
prosocial_train_chats

,match_id,player_name,player_chat,action
3593,18727,[B]rix,WP,1
2156,27123,#Ex3maL,gg,1
490,32764,Jin Kazama,GG,1
4107,4000,Bombur,?. 12 assist. nice stun. lion,1
3541,45327,Ignatize,I'm sorry,1
...,...,...,...,...
2208,2611,bloo,gg,1
3617,15677,coconutice,jug is the best in this team,1
817,9994,-| RompeLus |-,gg,1
4401,19325,Trickster,outplayed. gg,1


In [ ]:
sample = prosocial_train_chats[['player_chat', 'action']].sample(n=5).to_dict(orient='records')

In [233]:
instruction = (
    "You are a data generator tasked with creating realistic game chat content that is prosocial in nature."
    "You will be given some data that you should look to emulate by maintaining the style, slang, and tone of the chat, "
    "including typos where appropriate, but the content must not be copied. "
    "Return a list of new entries in valid JSON format, where each entry is a Python dictionary with the keys "
    "'player_chat' and 'action'."
)
input = (
    f"Here is a sample of labeled game chat data:\n\n"
    f"{prosocial_train_chats[['player_chat', 'action']].to_dict(orient='records')}\n\n"
    "Please generate 500 new samples that are similar in style, and output them as a valid JSON list of dictionaries. "
    "Each dictionary should have 'player_chat' as a string and 'action' as a string (either '1' or '-1')."
)

response = client.responses.create(
    model="gpt-4o",
    instructions=instruction,
    input=input
)

In [234]:
print(response.output_text)

```json
[
    {"player_chat": "ty all", "action": "1"},
    {"player_chat": "gg ez", "action": "1"},
    {"player_chat": "GG!", "action": "1"},
    {"player_chat": "Well played, guys!", "action": "1"},
    {"player_chat": "glhf everyone!", "action": "1"},
    {"player_chat": "nice play, lina", "action": "1"},
    {"player_chat": "sick moves!", "action": "1"},
    {"player_chat": "learned a lot, thx", "action": "1"},
    {"player_chat": "cool strat, wp", "action": "1"},
    {"player_chat": "that stun tho!", "action": "1"},
    {"player_chat": "great save!", "action": "1"},
    {"player_chat": "ggwp :D", "action": "1"},
    {"player_chat": ":D gg", "action": "1"},
    {"player_chat": "let's do this!", "action": "1"},
    {"player_chat": "gg wp guys", "action": "1"},
    {"player_chat": "commend jug", "action": "1"},
    {"player_chat": "not mad, wp lol", "action": "1"},
    {"player_chat": "first time pudge, gg", "action": "1"},
    {"player_chat": "nice combo!", "action": "1"},
    {"pl

In [235]:
resp = response.output_text[8:-4]

In [236]:
data = json.loads(resp)
new_df = pd.DataFrame(data)

In [237]:
prosocial_df = new_df

,player_chat,action
0,ty all,1
1,gg ez,1
2,GG!,1
3,"Well played, guys!",1
4,glhf everyone!,1
...,...,...
282,slay them,1
283,NO U,1
284,incredible,1
285,peng gang,1


In [241]:
prosocial_test_df = test_df[test_df['action'] == 1]

In [243]:
X_test = np.array(sentence_transformer.encode(prosocial_test_df['player_chat'].to_list()))

In [245]:
X_test.shape

(393, 768)

In [249]:
gen_chat = np.array(sentence_transformer.encode(new_df['player_chat'].to_list()))

In [251]:
gen_chat.shape

(287, 768)

In [266]:
prosocial_gen_rbf = report_MMD(gen_chat, X_test, normalize=True)[0]
print(prosocial_gen_rbf)

0.33406870299543145


In [256]:
new_df.to_csv('../data/generated/class_seperated/prosocial_generated.csv', index=False)

In [265]:
p_value, x_bar, std, permuted = p_test(gen_chat, X_test, prosocial_gen_rbf)
print(p_value)
print(f'{x_bar} ± {std}')

100%|██████████| 1000/1000 [00:08<00:00, 123.02it/s]

0.0
0.011091943163167957 ± 0.0


# **Not-Prosocial Category Generations**

In [269]:
not_prosocial_train_chats = train_df[train_df['action'] == -1]

In [270]:
instruction = (
    "You are a data generator tasked with creating realistic game chat content that is not prosocial in nature."
    "You will be given some data that you should look to emulate by maintaining the style, slang, and tone of the chat, "
    "including typos where appropriate, but the content must not be copied. "
    "Return a list of new entries in valid JSON format, where each entry is a Python dictionary with the keys "
    "'player_chat' and 'action'."
)
input = (
    f"Here is a sample of labeled game chat data:\n\n"
    f"{prosocial_train_chats[['player_chat', 'action']].to_dict(orient='records')}\n\n"
    "Please generate 500 new samples that are similar in style, and output them as a valid JSON list of dictionaries. "
    "Each dictionary should have 'player_chat' as a string and 'action' as a string (either '1' or '-1')."
)

response = client.responses.create(
    model="gpt-4o",
    instructions=instruction,
    input=input
)

In [272]:
print(response.output_text)

Sure, here is a list of 500 new samples:

```json
[
    {"player_chat": "gg. report noob team", "action": "-1"},
    {"player_chat": "bottom no def lol", "action": "-1"},
    {"player_chat": "ez af. uninstall pls", "action": "-1"},
    {"player_chat": "free win haha gg", "action": "-1"},
    {"player_chat": "trash supports wtf", "action": "-1"},
    {"player_chat": "easy mid lol", "action": "-1"},
    {"player_chat": "rofl this mid", "action": "-1"},
    {"player_chat": "waste time gg", "action": "-1"},
    {"player_chat": "noob team end fast", "action": "-1"},
    {"player_chat": "ggwp bots", "action": "-1"},
    {"player_chat": "LOL feeding much", "action": "-1"},
    {"player_chat": "ez pz team", "action": "-1"},
    {"player_chat": "cyka blyat lmao", "action": "-1"},
    {"player_chat": "u guys suck", "action": "-1"},
    {"player_chat": "afk noob gg", "action": "-1"},
    {"player_chat": "nice feed", "action": "-1"},
    {"player_chat": "bad players omg", "action": "-1"},
    {"pl

In [274]:
response.output_text[50]

'['

In [276]:
resp = response.output_text[50:-4]

In [277]:
data = json.loads(resp)
new_df = pd.DataFrame(data)

In [279]:
not_prosocial_df = new_df

In [280]:
not_prosocial_test_df = test_df[test_df['action'] == -1]

In [281]:
X_test = np.array(sentence_transformer.encode(prosocial_test_df['player_chat'].to_list()))

In [282]:
gen_chat = np.array(sentence_transformer.encode(new_df['player_chat'].to_list()))

In [283]:
not_prosocial_gen_rbf = report_MMD(gen_chat, X_test, normalize=True)[0]
print(not_prosocial_gen_rbf)

0.721097166228667


In [284]:
new_df.to_csv('../data/generated/class_seperated/not_prosocial_generated.csv', index=False)

In [285]:
p_value, x_bar, std, permuted = p_test(gen_chat, X_test, not_prosocial_gen_rbf)
print(p_value)
print(f'{x_bar} ± {std}')

100%|██████████| 1000/1000 [00:11<00:00, 85.84it/s]

0.0
0.013293034154850643 ± 0.0
